<a href="https://colab.research.google.com/github/December04/MachineLearningProgramming/blob/main/Week10/Assignment02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [17]:
DATASET_PATH = "/content/drive/MyDrive/Colab Notebooks/UCI HAR Dataset/UCI HAR Dataset"

In [18]:
def load_file(filepath):
    return np.loadtxt(filepath)

def load_group(prefix, group="train"):
    signal_names = [
        "body_acc_x_", "body_acc_y_", "body_acc_z_",
        "body_gyro_x_", "body_gyro_y_", "body_gyro_z_",
        "total_acc_x_", "total_acc_y_", "total_acc_z_"
    ]
    data_dir = os.path.join(prefix, group, "Inertial Signals")
    arrays = []
    for s in signal_names:
        path = os.path.join(data_dir, f"{s}{group}.txt")
        arr = load_file(path)
        arrays.append(arr)
    stacked = np.stack(arrays, axis=-1)
    return stacked

def load_labels(prefix, group="train"):
    path = os.path.join(prefix, group, f"y_{group}.txt")
    y = load_file(path).astype(int).flatten()
    return y - 1

In [19]:
X_train = load_group(DATASET_PATH, "train")
X_test  = load_group(DATASET_PATH, "test")
y_train = load_labels(DATASET_PATH, "train")
y_test  = load_labels(DATASET_PATH, "test")

print("X_train shape:", X_train.shape)
print("X_test  shape:", X_test.shape)

X_train shape: (7352, 128, 9)
X_test  shape: (2947, 128, 9)


In [20]:
n_timesteps = X_train.shape[1]
n_features = X_train.shape[2]

In [21]:
scaler = StandardScaler()
X_train_2d = X_train.reshape(-1, n_features)
X_test_2d  = X_test.reshape(-1, n_features)

In [22]:
scaler.fit(X_train_2d)
X_train_scaled = scaler.transform(X_train_2d).reshape(-1, n_timesteps, n_features)
X_test_scaled  = scaler.transform(X_test_2d).reshape(-1, n_timesteps, n_features)

In [23]:
n_classes = len(np.unique(y_train))
y_train_cat = to_categorical(y_train, num_classes=n_classes)
y_test_cat  = to_categorical(y_test, num_classes=n_classes)

In [24]:
def build_lstm(input_shape, n_classes):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=False))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [25]:
input_shape = (n_timesteps, n_features)
model = build_lstm(input_shape, n_classes)
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,750 (92.77 KB)

 Trainable params: 23,622 (92.27 KB)

 Non-trainable params: 128 (512.00 B)

In [26]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
    ModelCheckpoint("best_har_lstm.h5", monitor='val_loss', save_best_only=True)
]

In [27]:
history = model.fit(
    X_train_scaled, y_train_cat,
    validation_split=0.2,
    epochs=60,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/60


92/92 - 26s - 281ms/step - accuracy: 0.4827 - loss: 1.4055 - val_accuracy: 0.7335 - val_loss: 0.9920
Epoch 2/60


92/92 - 12s - 132ms/step - accuracy: 0.6791 - loss: 0.8037 - val_accuracy: 0.8103 - val_loss: 0.6542
Epoch 3/60


92/92 - 7s - 72ms/step - accuracy: 0.7789 - loss: 0.5234 - val_accuracy: 0.8239 - val_loss: 0.4278
Epoch 4/60


92/92 - 8s - 89ms/step - accuracy: 0.8181 - loss: 0.4143 - val_accuracy: 0.8960 - val_loss: 0.3722
Epoch 5/60


92/92 - 9s - 94ms/step - accuracy: 0.8925 - loss: 0.2953 - val_accuracy: 0.9171 - val_loss: 0.2369
Epoch 6/60
92/92 - 9s - 97ms/step - accuracy: 0.9131 - loss: 0.2783 - val_accuracy: 0.9137 - val_loss: 0.2985
Epoch 7/60
92/92 - 7s - 71ms/step - accuracy: 0.9260 - loss: 0.2342 - val_accuracy: 0.8484 - val_loss: 0.4126
Epoch 8/60


92/92 - 10s - 110ms/step - accuracy: 0.9342 - loss: 0.1994 - val_accuracy: 0.9123 - val_loss: 0.2360
Epoch 9/60


92/92 - 8s - 90ms/step - accuracy: 0.9478 - loss: 0.1500 - val_accuracy: 0.9205 - val_loss: 0.2266
Epoch 10/60
92/92 - 6s - 70ms/step - accuracy: 0.9510 - loss: 0.1337 - val_accuracy: 0.9035 - val_loss: 0.2285
Epoch 11/60
92/92 - 9s - 93ms/step - accuracy: 0.9475 - loss: 0.1352 - val_accuracy: 0.9205 - val_loss: 0.2793
Epoch 12/60
92/92 - 7s - 78ms/step - accuracy: 0.9524 - loss: 0.1387 - val_accuracy: 0.9184 - val_loss: 0.2886
Epoch 13/60
92/92 - 10s - 111ms/step - accuracy: 0.9344 - loss: 0.1945 - val_accuracy: 0.8688 - val_loss: 0.5126
Epoch 14/60
92/92 - 12s - 129ms/step - accuracy: 0.9206 - loss: 0.2506 - val_accuracy: 0.9062 - val_loss: 0.2938
Epoch 15/60
92/92 - 10s - 105ms/step - accuracy: 0.9413 - loss: 0.1452 - val_accuracy: 0.9130 - val_loss: 0.3187
Epoch 16/60
92/92 - 10s - 104ms/step - accuracy: 0.9541 - loss: 0.1231 - val_accuracy: 0.9096 - val_loss: 0.3333
Epoch 17/60
92/92 - 7s - 78ms/step - accuracy: 0.9507 - loss: 0.1231 - val_accuracy: 0.8960 - val_loss: 0.3632


In [28]:
loss, acc = model.evaluate(X_test_scaled, y_test_cat, verbose=0)
print(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")

Test loss: 0.3447, Test accuracy: 0.9067


In [29]:
y_pred_prob = model.predict(X_test_scaled)
y_pred = np.argmax(y_pred_prob, axis=1)
print("Classification report:")
print(classification_report(y_test, y_pred, digits=4))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step
Classification report:
              precision    recall  f1-score   support

           0     0.9723    0.9194    0.9451       496
           1     0.9638    0.9618    0.9628       471
           2     0.9119    0.9857    0.9474       420
           3     0.8020    0.8330    0.8172       491
           4     0.8052    0.8083    0.8068       532
           5     1.0000    0.9497    0.9742       537

    accuracy                         0.9067      2947
   macro avg     0.9092    0.9096    0.9089      2947
weighted avg     0.9088    0.9067    0.9073      2947

Confusion matrix:
[[456   6  34   0   0   0]
 [ 12 453   6   0   0   0]
 [  0   6 414   0   0   0]
 [  0   3   0 409  79   0]
 [  1   0   0 101 430   0]
 [  0   2   0   0  25 510]]
